# Lab | Avila Bible 

In this lab, we will explore the [**Avila Bible dataset**](https://archive.ics.uci.edu/ml/datasets/Avila) which has been extracted from 800 images of the 'Avila Bible', an XII century giant Latin copy of the Bible. The prediction task consists in associating each pattern to a copyist. You will use supervised learning algorithms to figure out what feature patterns each copyist are likely to have and use our model to predict those copyist.

-----------------------------------------------------------------------------------------------------------------

## Before your start:
    - Read the README.md file,
    - Comment as much as you can and use the APIla-bible in the README.md,
    - Happy learning!

In [1]:
# Import your libraries
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


Bad key "text.kerning_factor" on line 4 in
/Users/miriam/miniconda3/envs/analytics_env/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


![machine-learning](https://miro.medium.com/proxy/1*halC1X4ydv_3yHYxKqvrwg.gif)

The Avila data set has been extracted from 800 images of the the **Avila Bible**, a giant Latin copy of the whole Bible produced during the XII century between Italy and Spain. The palaeographic analysis of the  manuscript has  individuated the presence of 12 copyists. The pages written by each copyist are not equally numerous. 
Each pattern contains 10 features and corresponds to a group of 4 consecutive rows.

## What am I expected to do?

Well, your prediction task consists in associating each pattern to one of the 8 monks we will be evaluating (labeled as:  Marcus, Clarius, Philippus, Coronavirucus, Mongucus, Paithonius, Ubuntius, Esequlius). For that aim, you should: 
- Train a minimum of 4 different models
- Perform a minimum of 4 Feature Extraction and Engineering techniques
- Must contain a summary of the machine learning tools and algorithms
- and the results or the score obtained with each of them

You won't get much more instructions from now on. Remember to comment your code as much as you can. Keep the requirements in mind and have fun! 

Just one last piece of advice, take a moment to explore the data, remember this dataset contains two files: **train** and **test**. You will find both files in `data` folder. The **test** files contains the data you will predict for, therefore it does not include the labels.
Use the **train** dataset as you wish, but don't forget to split it into **train** and **test** again so you can evaluate your models. Just be sure to train it again with the whole data before predicting.
We have also included a **sample submission** which is of the exact shape and format you must use when evaluating your predictions against the groundtruth through the `APIla-bible`. It won't work unless it is the exact same shape. 



#### Train dataset

In [2]:
train_dataset = pd.read_csv('../data/training_dataset.csv', index_col=0)

In [3]:
train_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.241386,0.109171,-0.127126,0.380626,0.172340,0.314889,0.484429,0.316412,0.188810,0.134922,Marcus
1,0.303106,0.352558,0.082701,0.703981,0.261718,-0.391033,0.408929,1.045014,0.282354,-0.448209,Clarius
2,-0.116585,0.281897,0.175168,-0.152490,0.261718,-0.889332,0.371178,-0.024328,0.905984,-0.877830,Philippus
3,-0.326430,-0.652394,0.384996,-1.694222,-0.185173,-1.138481,-0.232828,-1.747116,-1.183175,-0.807380,Philippus
4,-0.437525,-0.471816,0.463236,-0.545248,0.261718,-0.972381,0.824183,-3.108388,-2.991700,-1.141030,Philippus


#### Test dataset


In [4]:
test_dataset = pd.read_csv('../data/test_dataset.csv', index_col=0)

In [5]:
test_dataset.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.017834,0.132725,0.125378,1.357345,0.261718,0.190314,0.182426,0.445253,-0.715453,0.189796
1,-0.202992,-0.000745,-3.210528,-0.527256,0.082961,0.771662,0.144676,0.098572,0.251173,0.745333
2,1.019049,0.211237,-0.155578,-0.311855,0.261718,0.107265,0.484429,0.339303,-0.310094,-0.049630
3,0.451232,-0.267686,0.335206,-0.831336,0.261718,0.024215,0.220177,0.988787,0.032902,0.025485
4,-0.227680,0.109171,0.413447,0.118917,0.172340,0.480988,0.522180,0.091562,0.313536,0.256389


#### Sample submission

In [6]:
sample_submission = pd.read_csv('../data/sample_submission.csv', header=None, index_col=0)

In [7]:
sample_submission.head()

,1
0,
0,Philippus
1,Ubuntius
2,Esequlius
3,Coronavirucus
4,Philippus


`Keep calm and code on!`

# Challenge - train your models, make the best prediction

In [8]:
#train_dataset.isnull().sum() #No hay nulos

In [9]:
#train_dataset.hist(figsize=[10,10]) #Distribucion features

### Problem type

In [10]:
#PROBLEMA DE CLASIFICACIÓN MULTICLASE

### Variables 

In [11]:
X = train_dataset.drop(columns='10').values
y = train_dataset['10'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9012, 10)
(9012,)
(3005, 10)
(3005,)


### Logistic Regression

In [12]:
lr = LogisticRegression(max_iter=200)
lr_trained = lr.fit(X_train, y_train)

### Random Forest

In [13]:
rf = RandomForestClassifier()
rf_trained = rf.fit(X_train, y_train)

### SVM

In [14]:
svm = SVC()
svm_trained = svm.fit(X_train, y_train)

### kNN

In [15]:
knn = KNeighborsClassifier()
knn_trained = knn.fit(X_train, y_train)

### Individual Report

In [16]:
models_train={'Logistic Regression':lr_trained,
        'Random Forest':rf_trained,
        'SVM':svm_trained,
        'kNN':knn_trained}

for name, model in models_train.items():
    y_pred = model.predict(X_test)
    print(f"MODEL -> {name}")
    print(classification_report(y_test,y_pred))

MODEL -> Logistic Regression


/Users/miriam/miniconda3/envs/analytics_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

      Clarius       0.38      0.10      0.15       591
Coronavirucus       0.84      0.87      0.85       252
    Esequlius       0.00      0.00      0.00       110
       Marcus       0.54      0.94      0.69      1280
     Mongucus       0.77      0.77      0.77       155
   Paithonius       0.53      0.18      0.26       154
    Philippus       0.53      0.29      0.37       330
     Ubuntius       0.00      0.00      0.00       133

     accuracy                           0.57      3005
    macro avg       0.45      0.39      0.39      3005
 weighted avg       0.50      0.57      0.49      3005

MODEL -> Random Forest
               precision    recall  f1-score   support

      Clarius       0.99      0.97      0.98       591
Coronavirucus       1.00      1.00      1.00       252
    Esequlius       1.00      0.99      1.00       110
       Marcus       0.98      1.00      0.99      1280
     Mongucus       1.00      0.99    

### Cross Validation

In [17]:
models_inst={'Logistic Regression':lr,
        'Random Forest':rf,
        'SVM':svm,
        'kNN':knn}

for name, model in models_inst.items():
    scores = cross_val_score(model, X, y, cv=5)
    print(f"({name}) acccuracy={np.mean(scores)}")

(Logistic Regression) acccuracy=0.5688614412239831
(Random Forest) acccuracy=0.9871024018091639
(SVM) acccuracy=0.6911890156716196
(kNN) acccuracy=0.7442793706978866


### Train definitive model

In [18]:
rf = rf.fit(X, y)
X_test = test_dataset.values
y_pred=pd.DataFrame(rf.predict(X_test))
y_pred.head(5)

,0
0,Marcus
1,Esequlius
2,Marcus
3,Marcus
4,Esequlius


## What do I do once I have a prediction?

Once you have already trained your model and made a prediction with it, you are ready to check what is the accuracy of it. 

Save your prediction as a `.csv` file.

In [19]:
y_pred.to_csv('./final.csv', header = False)

Now you are ready to know the truth! Are you good enough to call yourself a pro?

Lucky you have the ultimate **APIla-bible** which give you the chance of checking the accuracy of your predictions as many times as you need in order to become the pro you want to be. 

## How do I post my prediction to the APIla-bible?

Easy peasy! You should only fulfil the path to your prediction `.csv` and run the cell below! 

In [20]:
my_submission = './final.csv'
with open(my_submission) as f:
    res = requests.post("http://apila-bible.herokuapp.com/check", files={"csv_data":f.read()})
res.json()

{'accuracy': 0.9935097353969047,
 'quote': "AWESOME! A-W-E-S-O-M-E! Amazing score!!! So cool! I can't even... But wait, maybe...too good to be true? Overfit much?",
 'tip': 'If you think you may have overfitted your model, visit http://apila-bible.herokuapp.com/check/overfit on your browser for some follow up. ;)'}

![hope-you-enjoy](https://imgs.xkcd.com/comics/machine_learning.png)